### LINK PREDICTION 

### import libraries 

In [3]:
#per creare rete
import pandas as pd
from rich import print
import networkx as nx
#import networkx.algorithms as nx
import pandas as pd
import matplotlib.pyplot as plt
from networkx.utils import np_random_state

#per link prediction
import linkpred
%matplotlib inline 

##per opinion diffusion
#import ndlib
#import networkx as nx
#import ndlib.models.ModelConfig as mc
#import ndlib.models.opinions as op

#altre
import seaborn as sns
import numpy as np
import math
import warnings
from collections import Counter
import numpy as np
warnings.filterwarnings('ignore')

In [15]:
G=nx.read_gexf('final_net.gexf')
df_edges=pd.read_csv('df_final_edges')
df_edges

,Unnamed: 0,source,target,w,id
0,0,290883,2956836471,0.022888,0
1,1,1471971,1205226529455632385,0.000000,2
2,2,2052911,1401015246718914560,0.066731,3
3,3,2052911,2202063265,0.046422,4
4,4,2052911,1317817827970134025,0.017730,5
...,...,...,...,...,...
11963,11963,1499698836117368833,1502720034971521026,0.000000,12762
11964,11964,1499737837620576256,1501600840628547594,0.010961,12763
11965,11965,1499737837620576256,1500461594005938179,0.008704,12764
11966,11966,1499737837620576256,1501482734392422402,0.006447,12765


In [6]:
G.number_of_edges()

11968

In [7]:
G.number_of_nodes()

10184

In [16]:
del df_edges['Unnamed: 0']
del df_edges['id']
del df_edges['w']

In [17]:
df_edges

,source,target
0,290883,2956836471
1,1471971,1205226529455632385
2,2052911,1401015246718914560
3,2052911,2202063265
4,2052911,1317817827970134025
...,...,...
11963,1499698836117368833,1502720034971521026
11964,1499737837620576256,1501600840628547594
11965,1499737837620576256,1500461594005938179
11966,1499737837620576256,1501482734392422402


## Unsupervised link prediction :  Neighborhood
Tries to relate neighborhoods of node pairs with the likelihood of observing a tie among them.

"How many friend we have to share in order to become friends?"
### U.1 Common Neighbors
The more friends we share, the more likely we will become friends.


In [9]:
cn = linkpred.predictors.CommonNeighbours(G, excluded=G.edges()) # We aim to predict only new links, thus we exclude existing ones
cn_results = cn.predict()

top = cn_results.top()
for edge, score in top.items():
    print(edge, score)

2202063265 - 1401015246718914560 49.0

2956836471 - 2616995078 47.0

27493883 - 1270133776748855304 34.0

27493883 - 1205226529455632385 32.0

2616995078 - 2202063265 32.0

1401015246718914560 - 1317817827970134025 30.0

790870659333259264 - 2202063265 29.0

820731335157383170 - 1401015246718914560 24.0

2616995078 - 1317817827970134025 24.0

790870659333259264 - 1317817827970134025 20.0

### U.2 Jaccard
The more similar our friends circles are, the more likely we will become friends.

In [10]:
jc = linkpred.predictors.Jaccard(G, excluded=G.edges())
jc_results = jc.predict()

top = jc_results.top(5)
for edge, score in top.items():
    print(edge, score)

999759383830679553 - 976625864157868033 1.0

999759383830679553 - 972826051 1.0

999759383830679553 - 956566580 1.0

999759383830679553 - 941286290 1.0

999759383830679553 - 92342478 1.0

### U.3 Adamic Adar
The more selective our mutual friends are, the more likely we will become friends.

In [22]:
aa = linkpred.predictors.AdamicAdar(G, excluded=G.edges())
aa_results = aa.predict()

top = aa_results.top(5)
for edge, score in top.items():
    print(edge, score)

2956836471 - 2616995078 42.28714215169167

27493883 - 1205226529455632385 40.72454159594907

27493883 - 1270133776748855304 37.27847191402833

2202063265 - 1401015246718914560 34.68565395845539

4018888580 - 27493883 23.68401507705543

## 2.C Evaluation and comparison of different predictors

To evaluate a link predictor we have to separate the network used for training from the one used for testing purposes.

In [25]:
import random
import itertools
from linkpred.evaluation import Pair
from sklearn.model_selection import train_test_split

x=df_edges['source']
y=df_edges['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
print(len(X_train))

In [ ]:
#random net per ROC curve 0.5
nodes=list(G.nodes)
universe = set([Pair(i) for i in itertools.product(nodes, nodes) if i[0]!=i[1]])

After that, we can apply the predictors to the training network

In [ ]:
cn = linkpred.predictors.CommonNeighbours(x_train, excluded=training.edges())
cn_results = cn.predict()

aa = linkpred.predictors.AdamicAdar(training, excluded=training.edges())
aa_results = aa.predict()

jc = linkpred.predictors.Jaccard(training, excluded=training.edges())
jc_results = jc.predict()

Now we can evaluate the obtained prediction against the test

In [ ]:
cn_evaluation = linkpred.evaluation.EvaluationSheet(cn_results, test, universe)
aa_evaluation = linkpred.evaluation.EvaluationSheet(aa_results, test, universe)
jc_evaluation = linkpred.evaluation.EvaluationSheet(jc_results, test, universe)

The results can be easily compared using a ROC plot

In [ ]:
plt.plot(cn_evaluation.fallout(), cn_evaluation.recall(), label="Common Neighbors")
plt.plot(aa_evaluation.fallout(), aa_evaluation.recall(), label="Adamic Adar")
plt.plot(jc_evaluation.fallout(), jc_evaluation.recall(), label="Jaccard")
plt.ylabel("TPR")
plt.xlabel("FPR")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import auc

print("Area Under Roc Curve (AUROC)")
print(f"Common Neigh.: \t {auc(cn_evaluation.fallout(), cn_evaluation.recall())}")
print(f"Adamic Adar: \t {auc(aa_evaluation.fallout(), aa_evaluation.recall())}")
print(f"Jaccard: \t {auc(jc_evaluation.fallout(), jc_evaluation.recall())}")

In [ ]:
OPEN QUESTION: DA MARKOV CHAIN RICAVIAMO PROB TRANSIZIONE CHE USIAMO PER LA SUPERVISED LINK PREDICION BAYESIAN!!!